### Imports...

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import random

%env CUDA_VISIBLE_DEVICES=0
import torch
import torchvision as tv
from torchvision import datasets, transforms

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#import torch_dip_utils as utils
import utils
import math

env: CUDA_VISIBLE_DEVICES=0


### Set up Hyperparameters, network filter and I/O sizes, and waveform parameters

In [3]:
#set up hyperparameters, net input/output sizes, and whether the problem is compressed sensing

LR = 1e-3 # learning rate
MOM = 0.9 # momentum
NUM_ITER = 100 # number iterations
WD = 1e-4 # weight decay for l2-regularization

Z_NUM = 32 # input seed
NGF = 64 # number of filters per layer
ALEX_BATCH_SIZE = 1 # batch size of gradient step
nc = 1 #num channels in the net I/0

#choose the number of samples and periods in the training waveform
WAVE_SIZE = 1024
WAVE_PERIODS = 2

### Choose whether the problem is compressed sensing or DIP

In [4]:
compressed = False

if compressed:
    num_measurements = 64
else:
    num_measurements = WAVE_SIZE

### Use CUDA if Possible

In [5]:
CUDA = torch.cuda.is_available()
print(CUDA)

#save the correct datatype depending on CPU or GPU execution
if CUDA : 
    dtype = torch.cuda.FloatTensor  
    print(torch.cuda.device(0))
else:
    dtype = torch.FloatTensor
    print("NO DEVICES")

True


### Create and plot the training and reference waveforms

In [6]:
#Produces a sinusoid with optional additive gaussian noise distributed (mean, std)
def get_sinusoid(num_samples, num_periods, noisy=True, std=0.1, mean=0):
    
    Fs = num_samples
    x = np.arange(num_samples)
    
    y = np.sin(2*np.pi * num_periods * x / Fs)
    
    if noisy:
        y += (std * np.random.randn(num_samples)) + mean
    
    return y

### Util function for normalizing noisy wave range to [-1,1] and renormalizing back to native range

In [7]:
def get_stats(x):
    a = np.min(x)
    b = np.max(x)
    mu = (a+b)/2.0
    sigma = (b-a)/2.0
    return [mu, sigma]

def normalise(x, mu, sigma):
    return (x-mu)/sigma

def renormalise(x, mu, sigma):
    return x*sigma + mu

### Prepare waveform for net training

In [8]:
#get the proper MSE loss based on the datatype
mse = torch.nn.MSELoss().type(dtype)

### Define the network architecture

In [9]:
class DCGAN(nn.Module):
    def __init__(self, nz, ngf=64, output_size=1024, nc=1, num_measurements=64):
        super(DCGAN, self).__init__()
        self.nc = nc
        self.output_size = output_size

        # Deconv Layers: (in_channels, out_channels, kernel_size, stride, padding, bias = false)
        # Inputs: R^(N x Cin x Lin), Outputs: R^(N, Cout, Lout) s.t. Lout = (Lin - 1)*stride - 2*padding + kernel_size

        self.conv1 = nn.ConvTranspose1d(nz, ngf, 4, 1, 0, bias=False)
        self.bn1 = nn.BatchNorm1d(ngf)
        # LAYER 1: input: (random) zϵR^(nzx1), output: x1ϵR^(64x4) (channels x length)

        self.conv2 = nn.ConvTranspose1d(ngf, ngf, 6, 2, 2, bias=False)
        self.bn2 = nn.BatchNorm1d(ngf)
        # LAYER 2: input: x1ϵR^(64x4), output: x2ϵR^(64x8) (channels x length)

        self.conv3 = nn.ConvTranspose1d(ngf, ngf, 6, 2, 2, bias=False)
        self.bn3 = nn.BatchNorm1d(ngf)
        # LAYER 3: input: x1ϵR^(64x8), output: x2ϵR^(64x16) (channels x length)

        self.conv4 = nn.ConvTranspose1d(ngf, ngf, 6, 2, 2, bias=False)
        self.bn4 = nn.BatchNorm1d(ngf)
        # LAYER 4: input: x1ϵR^(64x16), output: x2ϵR^(64x32) (channels x length)

        self.conv5 = nn.ConvTranspose1d(ngf, ngf, 6, 2, 2, bias=False)
        self.bn5 = nn.BatchNorm1d(ngf)
        # LAYER 5: input: x2ϵR^(64x32), output: x3ϵR^(64x64) (channels x length)

        self.conv6 = nn.ConvTranspose1d(ngf, ngf, 6, 2, 2, bias=False)
        self.bn6 = nn.BatchNorm1d(ngf)
        # LAYER 6: input: x3ϵR^(64x64), output: x4ϵR^(64x128) (channels x length)

        self.conv7 = nn.ConvTranspose1d(ngf, ngf, 6, 2, 2, bias=False)
        self.bn7 = nn.BatchNorm1d(ngf)
        # LAYER 7: input: x4ϵR^(64x128), output: x5ϵR^(64x256) (channels x length)

        self.conv8 = nn.ConvTranspose1d(ngf, ngf, 6, 2, 2, bias=False)
        self.bn8 = nn.BatchNorm1d(ngf)
        # LAYER 8: input: x5ϵR^(64x256), output: x6ϵR^(64x512) (channels x length)

        self.conv9 = nn.ConvTranspose1d(ngf, nc, 4, 2, 1, bias=False)  # output is image
        # LAYER 9: input: x6ϵR^(64x512), output: (sinusoid) G(z,w)ϵR^(1x1024) (channels x length)

        self.fc = nn.Linear(output_size * nc, num_measurements, bias=False)  # output is A; measurement matrix
        # each entry should be drawn from a Gaussian (random noisy measurements)
        # don't compute gradient of self.fc! memory issues

    def forward(self, x):
        input_size = x.size()
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = F.relu(self.bn7(self.conv7(x)))
        x = F.relu(self.bn8(self.conv8(x)))
        x = F.tanh(self.conv9(x))

        return x

    def measurements(self, x):
        # this gives the image - make it a single row vector of appropriate length
        y = self.forward(x).view(1, -1)
        y = y.cpu()

        # pass thru FC layer - returns A*image
        meas = self.fc(y)

        if CUDA:
            return meas.cuda()
        else:
            return meas

### Train the network while tracking loss vs. reference wave

In [10]:
num_samples = 20
period_list = [1, 4, 16, 64, 128, 256, 512]

mse_log = (1e6)*np.ones((len(period_list), num_samples))
iter_log = (1e6)*np.ones((len(period_list), num_samples))

for pd in range(len(period_list)):
    print(pd)
    for j in range(num_samples):
        
        # get a DCGAN that outputs images of size WAVE_SIZE
        net = DCGAN(Z_NUM,NGF,WAVE_SIZE,nc,num_measurements) # initialize network
        net.fc.requires_grad = False

        if CUDA: # move network to GPU if available
            net.cuda()

        if compressed:
            net.fc.weight.data = (1/math.sqrt(1.0*num_measurements)) * torch.randn(num_measurements, WAVE_SIZE*nc)
        else:
            net.fc.weight.data = torch.eye(num_measurements)

        allparams = [x for x in net.parameters()] #specifies which to compute gradients of
        allparams = allparams[:-1] # get rid of last item in list (fc layer) because it's memory intensive

        z = Variable(torch.zeros(ALEX_BATCH_SIZE*Z_NUM).type(dtype).view(ALEX_BATCH_SIZE,Z_NUM,1))
        z.data.normal_().type(dtype)

        # Define optimizer
        optim = torch.optim.RMSprop(allparams,lr=LR,momentum=MOM, weight_decay=WD)
        
        y0 = get_sinusoid(num_samples = WAVE_SIZE, num_periods = period_list[pd], noisy=True)
        y0_denoised = get_sinusoid(num_samples = WAVE_SIZE, num_periods = period_list[pd], noisy=False)
        
        MU = get_stats(y0)[0]
        SIGMA = get_stats(y0)[1]

        y = torch.Tensor(y0)
        y = normalise(y, MU, SIGMA)
        y = Variable(y.type(dtype))
        
        measurements = Variable(torch.mm(y.cpu().data.view(ALEX_BATCH_SIZE,-1),net.fc.weight.data.permute(1,0)),requires_grad=False) 

        if CUDA: # move measurements to GPU if possible
            measurements = measurements.cuda()
        
        for i in range(NUM_ITER):
            optim.zero_grad() # clears graidents of all optimized variables
            out = net(z) # produces wave (in form of data tensor) i.e. G(z,w)
    
            loss = mse(net.measurements(z),measurements) # calculate loss between AG(z,w) and Ay
         
            # DCGAN output is in [-1,1]. Renormalise to [0,1] before plotting
            wave = renormalise(out, MU, SIGMA).data[0].cpu().numpy()[0,:] 

            cur_mse = np.mean((y0_denoised - wave)**2)
            
            if (cur_mse <= mse_log[pd][j]):
                mse_log[pd][j] = cur_mse
                iter_log[pd][j] = i
    
            loss.backward()
            optim.step()

0


/home/sravula/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


1
2
3
4
5
6


In [11]:
print(iter_log)

[[68. 70. 52. 50. 71. 66. 86. 71. 77. 81. 56. 94. 58. 59. 82. 67. 77. 65.
  57. 66.]
 [67. 43. 47. 55. 74. 52. 56. 67. 62. 88. 74. 70. 59. 47. 47. 51. 44. 65.
  57. 60.]
 [67. 41. 83. 42. 71. 80. 52. 53. 52. 67. 48. 61. 71. 54. 71. 60. 50. 69.
  55. 67.]
 [60. 44. 60. 82. 67. 66. 56. 58. 70. 71. 49. 69. 47. 67. 81. 63. 52. 65.
  68. 62.]
 [44. 50. 52. 44. 45. 65. 43. 45. 52. 32. 54. 70. 58. 59. 55. 47. 70. 60.
  60. 44.]
 [33. 36. 48. 27. 55. 95. 41. 75. 46. 43. 34. 38. 48. 52. 61. 48. 49. 44.
  45. 47.]
 [18. 15.  7.  9. 18.  6.  6. 22. 13.  9. 21. 12. 22. 15. 13.  8. 11.  6.
  15. 11.]]


In [12]:
mean_mse = np.around(np.mean(mse_log, axis = 1), 5)
mean_iter = np.around(np.mean(iter_log, axis = 1), 1)
std_iter = np.around(np.std(iter_log, axis = 1), 1)

In [13]:
print("Periodicities: ", period_list)
print("Mean MSE: ", mean_mse)
print("Mean best iteration: ", mean_iter)
print("STD best iteration: ", std_iter)

Periodicities:  [1, 4, 16, 64, 128, 256, 512]
Mean MSE:  [0.00101 0.00103 0.00093 0.00071 0.00086 0.00108 0.00049]
Mean best iteration:  [68.6 59.2 60.7 62.8 52.4 48.2 12.8]
STD best iteration:  [11.4 11.5 11.6  9.8  9.6 14.8  5.2]
